# Scaling QCD Jets with Coffea Processor

In [ ]:
import awkward as ak
import numpy as np
import coffea
import uproot
import hist
import vector
from coffea import util, processor
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema, BaseSchema
from distributed.diagnostics.plugin import UploadDirectory
import matplotlib.pyplot as plt
from collections import defaultdict
import os
import pickle

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from smp_utils import *
from histogram_lib import *
from histogram_maker import *

In [ ]:
#qcd_jets_maker()
qcd_jets_maker(prependstr = "root://cmsxrootd.fnal.gov/")

In [ ]:
with open("pkl_files/QCD_pt_response_NEW.pkl", "rb") as f:
    out = pickle.load(f)
    
for k,v in out.items():
    print(k)
    #print(f"{v}\n")

In [ ]:
out["responses_histogram"].project("dataset", "frac").plot(density=True)